In [1]:
%timeit
import numpy as np
from scipy.spatial.distance import cdist

# Obstacle Heuristic Calculation Optimization

In [2]:
### Obstacles generation
min_x = -10
max_x = 10
min_y = -10
max_y = 10
num_obstacles = 100

obs_x = np.random.uniform(low=min_x, high=max_x, size=(num_obstacles,))
obs_y = np.random.uniform(low=min_y, high=max_y, size=(num_obstacles,))
obstacles = np.array([obs_x, obs_y]).T

# Obstacle dist tolerance
obstacle_dist_tol = 4

### Trajectory generation
N_TIMESTEPS = 60
trajectory = np.random.rand(N_TIMESTEPS, 5)
trajectory[:,0] = np.linspace(0, 20, N_TIMESTEPS)
trajectory[:,1] = np.linspace(0, 20, N_TIMESTEPS)

### Robot Stuff
robot_state = np.zeros(5)
robot_radius = 0.2

### Goal Stuff
goal_pose = np.array([4,4,np.pi/2])

In [3]:
trajectory[0:5, 0:3]

array([[0.        , 0.        , 0.47106082],
       [0.33898305, 0.33898305, 0.15251592],
       [0.6779661 , 0.6779661 , 0.31077507],
       [1.01694915, 1.01694915, 0.57546085],
       [1.3559322 , 1.3559322 , 0.38970894]])

### Nearest Obstacle Calculation

In [4]:
%%time
robot_pos = robot_state[0:2]
nearest_obstacles = []
for obs in obstacles:
    # Distance from robot pose to obstacle
    euclidean_dist_to_obs = np.linalg.norm(robot_pos - obs)

    # If robot is within range of the obstacle
    if euclidean_dist_to_obs < obstacle_dist_tol:
        nearest_obstacles.append(obs)
nearest_obstacles = np.array(nearest_obstacles)
        
print("Nearest obstacles: \n", nearest_obstacles, "\n")

Nearest obstacles: 
 [[ 3.71097963  1.05339285]
 [-0.80475161  1.55102777]
 [ 2.85552118  1.93380471]
 [-2.72564483  2.59704936]
 [ 3.0647341  -0.42515997]
 [-0.31646281 -2.4135895 ]
 [-0.02325552 -2.11067858]
 [-1.0391162  -0.98059357]
 [ 3.54929554 -0.03424242]
 [ 2.08113773 -3.35088509]
 [ 2.63836206 -0.05999882]] 

CPU times: user 12.7 ms, sys: 30.9 ms, total: 43.6 ms
Wall time: 1.87 ms


In [5]:
%%time
robot_pos = robot_state[0:2]
nearest_obstacles_ind = np.where(np.linalg.norm(robot_pos - obstacles, axis=1) < obstacle_dist_tol)

nearest_obstacles = obstacles[nearest_obstacles_ind]

print("Nearest obstacles: \n", nearest_obstacles, "\n")

Nearest obstacles: 
 [[ 3.71097963  1.05339285]
 [-0.80475161  1.55102777]
 [ 2.85552118  1.93380471]
 [-2.72564483  2.59704936]
 [ 3.0647341  -0.42515997]
 [-0.31646281 -2.4135895 ]
 [-0.02325552 -2.11067858]
 [-1.0391162  -0.98059357]
 [ 3.54929554 -0.03424242]
 [ 2.08113773 -3.35088509]
 [ 2.63836206 -0.05999882]] 

CPU times: user 7.86 ms, sys: 27.5 ms, total: 35.4 ms
Wall time: 1.55 ms


### Obstacle Cost Calculation

In [6]:
%%time
cost = 0.0
min_dist = np.inf   # Minimum distance to obstacle

# For every trajectory point 
for i in range(trajectory.shape[0]):
    for obs in nearest_obstacles:
        # Extract the trajectory's position
        trajectory_pos = trajectory[i, 0:2]

        # Calculate the distance between the obstacle and the trajectory's position
        euclidean_dist_to_obs = np.linalg.norm(trajectory_pos - obs)

        # Collision with obstacle? 
        # Note if dist between robot position and obstacle point is less than the radius of the robot
        # Then the obstacle point is within the robot (i.e collision)


        if euclidean_dist_to_obs <= robot_radius:
            cost = np.inf
            break
            
        if euclidean_dist_to_obs < min_dist:
            min_dist = euclidean_dist_to_obs
            
if cost != np.inf:
    cost = 1.0 / min_dist
            
print("min_dist: ", min_dist)
print("obs_cost: ", cost, "\n")

min_dist:  0.6524795029418083
obs_cost:  1.5326151940273063 

CPU times: user 35.1 ms, sys: 111 ms, total: 146 ms
Wall time: 6.38 ms


In [7]:
%%time
cost = 0.0
min_dist = np.inf   # Minimum distance to obstacle

trajectory_positions = trajectory[:, 0:2]
obs_x = nearest_obstacles[:,0]
obs_y = nearest_obstacles[:,1]


dx = trajectory_positions[:, 0] - obs_x[:, None]
dy = trajectory_positions[:, 1] - obs_y[:, None]
euclidean_dist = np.hypot(dx, dy)

if np.array(euclidean_dist <= robot_radius).any():
    cost = np.inf
else:
    min_dist = np.min(euclidean_dist)
    cost = 1.0 / min_dist

# for traj in trajectory_positions:
#     if (np.linalg.norm(traj - nearest_obstacles, axis=1) <= robot_radius).any():
#         cost = np.inf
#         break
    
    
print("min_dist: ", min_dist)
print("obs_cost: ", cost, "\n")

min_dist:  0.6524795029418083
obs_cost:  1.5326151940273063 

CPU times: user 0 ns, sys: 20.1 ms, total: 20.1 ms
Wall time: 882 µs


# Goal Heuristic Optimization

In [35]:
%%time
def calc_goal_heuristic(trajectory, goal_pose):
    ### Calculation of euclidean heuristic
    goal_pose_x, goal_pose_y, goal_pose_theta = goal_pose
    traj_end_x, traj_end_y, traj_end_theta, _, _ = trajectory[-1]

    # Magnitude calculations
    goal_mag = np.sqrt(goal_pose_x**2 + goal_pose_y**2)
    traj_end_mag = np.sqrt(traj_end_x**2 + traj_end_y**2)

    # Delta
    delta_x = goal_pose_x - traj_end_x
    delta_y = goal_pose_y - traj_end_y

    # Dot product between trajectory end and goal pose
    dot_product = (goal_pose_x * traj_end_x) + (goal_pose_y * traj_end_y)
    error_cos_theta = dot_product / (goal_mag * traj_end_mag + np.finfo(np.float32).eps)
    error_angle = np.arccos(error_cos_theta) 


    # ### Orientation error
    # error_angle = np.arctan2(delta_y, delta_x) - traj_end_theta

    ### Euclidean istance
    euclidean_dist_cost = np.sqrt((goal_pose_x - traj_end_x)**2 + (goal_pose_y - traj_end_y)**2)

    cost = error_angle + euclidean_dist_cost

    return cost

goal_cost = calc_goal_heuristic(trajectory, goal_pose)
print("goal cost: ", goal_cost, "\n")


goal cost:  22.627455599996345 

CPU times: user 320 µs, sys: 35 µs, total: 355 µs
Wall time: 277 µs


In [36]:
%%time
def calc_goal_heuristic(trajectory, goal_pose):
    ### Extract positions
    goal_pos = goal_pose[0:2]
    traj_pos = trajectory[-1, 0:2]  # Only considering end trajectory for goal heuristic
    
    ### Calculation of angle between goal and trajectory vectors
    goal_unit_vec = goal_pos / np.linalg.norm(goal_pos)
    traj_unit_vec = traj_pos / np.linalg.norm(traj_pos)
    traj_dot_goal = np.dot(goal_unit_vec, traj_unit_vec)
    
    error_angle = np.arccos(traj_dot_goal)
    
    ### Euclidean cost
    euclidean_dist_cost = np.linalg.norm(goal_pos - traj_pos)
    
    ### Total cost
    cost = error_angle + euclidean_dist_cost
    
    return cost

goal_cost = calc_goal_heuristic(trajectory, goal_pose)
print("goal cost: ", goal_cost, "\n")



goal cost:  22.627417019042948 

CPU times: user 214 µs, sys: 0 ns, total: 214 µs
Wall time: 220 µs


## Calculation of Search Space

In [10]:
dt = 0.2

v_a = 3
omega_a = 2

max_acc = 2
max_omega_acc = 3

v_min = 0
v_max = 5
omega_min = 0
omega_max = 5


In [11]:
%timeit max(v_max, v_a + max_acc * dt)

174 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [12]:
%timeit np.maximum(v_max, v_a + max_acc * dt)

1.42 µs ± 8.73 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
